In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_clever.csv to train_clever (1).csv
Saving test_clever.csv to test_clever (1).csv


In [ ]:
import pandas as pd

train = pd.read_csv("train_clever.csv")
test = pd.read_csv("test_clever.csv")
test

,ID,Question
0,11083,В каком году основан Санкт-Петербург?
1,18349,Какой город Золотого кольца России-город худож...
2,11374,Каким цветом написана надпись магазина «пятеро...
3,9699,Сколько весит пирамида Хеопса?
4,17748,В какой стране киберспорт впервые признали вид...
...,...,...
9995,28782,В каком году началась Первая Мировая война?
9996,13015,Самое популярное женское имя
9997,26647,Сколько на данный момент снято официальных фил...
9998,9578,У какого айфона появился впервые появился Touc...


# Baseline solution

In [ ]:
# Посмотрим размерность данных
print(train.shape)
print(test.shape)

(20000, 3)
(10000, 2)


In [ ]:
train.head()

,ID,Answer,Question
0,16019,0,Крупнейшим производителем кукурузы является:
1,5121,0,Кто первый в РОССИЮ завёз картошку ?
2,17169,0,Кто из мореплавателей сделал первое кругосветн...
3,15480,0,Как зовут героя серии романов Бариса Акунина?
4,3312,0,Какая сборная выйграла чемпионат мира по футбо...


Одна из первых концепций обрабтки естественных языков Bag-of-Words — это статистический анализ, анализирующий количественное вхождение слов в документах. Не смотря на то, что подход сформирован весьма давно, он отлично подходит для первичной обработки текстов и быстрого прототипирования.

Модуль CountVectorizer в sklearn как раз подзволяет сконвертировать набор текстов в матрицу токенов, находящихся в тексте.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Для векторизации текста будем использовать простой CountVectorizer
count_vectorizer = CountVectorizer()
# таким образом будет подсчитана следующая структура:
#        | слово1 | слово2 | слово3 | слово4
# текст1 |   1    |    1   |   1    |   0
# текст2 |   0    |    1   |   1    |   0
# текст3 |   2    |    1   |   0    |   0
# текст4 |   0    |    0   |   0    |   1

list_corpus = train["Question"].tolist()
list_labels = train["Answer"].tolist()
test_corpus = test["Question"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts = count_vectorizer.transform(test_corpus)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')

    roc_auc = roc_auc_score(y_test, y_predicted_counts)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)

    return accuracy, precision, recall, f1, roc_auc

accuracy, precision, recall, f1, roc_auc = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f, roc_auc = %.3f" % (accuracy, precision, recall, f1, roc_auc))

accuracy = 0.890, precision = 0.811, recall = 0.890, f1 = 0.846, roc_auc = 0.498


In [ ]:
y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)

y_test_pred

,index,Answer
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,0


In [ ]:
y_test_pred.to_csv('submition_test_Adelya.csv', index=False)
files.download("submition_test_Adelya.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>